In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt 
from sklearn.tree import export_graphviz
import pydot
from sklearn.tree import export_graphviz
import pydot
%matplotlib inline
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn import linear_model 
import statsmodels.api as sm
from sklearn.metrics import r2_score
import math as m
from pandas import DataFrame



/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%pwd
%ls '/content/drive/My Drive/Machine Learning Final'

 allfeatures_final.csv
 allfeatures_final.gsheet
 allfeatures_num_OH.csv
'Before_HepC Random Forest Regression Model_LBR.ipynb'
'Big Dreams.ipynb'
 Data/
 DATACHALLENGE.ipynb
 DecisionTreeCode.ipynb
 Geller.mutation.rates_update.csv
 GrabFeaturesCode.py
 H77_metadata.csv
 H77_metadata.gsheet
 HCV1a_TsMutFreq_195.csv
 HCV1a_TsMutFreq_195.gsheet
'HepC Random Forest Regression Model.ipynb'
'Histogram of Natural log of costs.png'
 one_hot_features.py
 RNAStructure_Conserved.csv
 RNAStructure_Conserved.gsheet


In [5]:
#pos_muts = pd.read_csv('/content/drive/My Drive/Machine Learning Final/H77_metadata.csv')
#freqs = pd.read_csv('/content/drive/My Drive/Machine Learning Final/HCV1a_TsMutFreq_195.csv')
#mut_rate = pd.read_csv('/content/drive/My Drive/Machine Learning Final/Geller.mutation.rates_update.csv')
#allfeatures_num_OH_OG = pd.read_csv('/content/drive/My Drive/Machine Learning Final/allfeatures_num_OH.csv') 
#logCosts = pd.read_csv('/content/drive/My Drive/Machine Learning Final/logCosts.csv') 
#allfeatures_num_OH = pd.concat([allfeatures_num_OH_OG,logCosts],axis=1,sort=False)

#allfeatures_num_OH.head()

features = pd.read_csv('/content/drive/My Drive/Machine Learning Final/allfeatures_final.csv') 



In [6]:
features.head()

,pos,makesCpG,bigAAChange,Avg_Mutation_Freq,Mutation_Rate,Cost,Positive AA,Negative AA,Hydrophobic AA,Polar AA,Nonpolar AA,5' UTR,Core,E1,E2,HVR1,NS1,NS2,NS3,NS4A,NS4B,NS5A,NS5B
0,264,0,0,0.183585,0.000011,0.000062,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,265,1,1,0.001653,0.000011,0.006840,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
2,266,0,0,0.000053,0.000003,0.059900,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,267,0,1,0.000089,0.000003,0.035500,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
4,268,0,1,0.000069,0.000003,0.045800,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Labels are the values we want to predict
labels=[]
for num in features['Cost']:
    ln = m.log(num)
    labels.append(ln)
labels_df = DataFrame(labels, columns=['ln_Cost'])

labels = np.array(labels_df['ln_Cost'])


# Remove the labels from the features
# axis 1 refers to the columns
features= features.drop('Cost', axis = 1)
cpg = features['makesCpG']
otherfeats = features.iloc[:,6:]
feats = pd.concat([cpg,otherfeats],axis=1,sort=False)
# Saving feature names for later use
feature_list = list(feats.columns)
# Convert to numpy array
features = np.array(feats)

In [ ]:
#TRAINING AND TESTING SETS
# Using Skicit-learn to split data into training and testing sets
#from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [ ]:
#look at the shape of all the data to make sure we did everything correct
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (6280, 17)
Training Labels Shape: (6280,)
Testing Features Shape: (2094, 17)
Testing Labels Shape: (2094,)


It looks as if everything is in order! 
Just to recap, to get the data into a form acceptable for machine learning we:


1.   One-hot encoded categorical variables
2.   Split data into features and labels
3.   Converted to arrays
4.   Split data into training and testing sets





**Establish Baseline**
Before we can make and evaluate predictions, we need to establish a baseline, a sensible measure that we hope to beat with our model. 
If our model cannot improve upon the baseline, then it will be a failure and we should try a different model or admit that machine learning is not right for our problem. 
The baseline prediction for our case can be the historical HIGHEST COST averages. In other words, our baseline is the error we would get if we simply predicted the 
average HIGHEST COST for all FEATURES.

In [ ]:
#Did not run this section of code, I think we need to use average cost values for each feature that Lorena calculated -Madu
# The baseline predictions are the historical averages
baseline_preds = test_features[:, feature_list.index('Avg_Mutation_Freq')]
# Baseline errors, and display average baseline error
baseline_errors = abs(baseline_preds - test_labels)
print('Average baseline error: ', round(np.mean(baseline_errors), 4))


ValueError: ignored

We now have our goal! If we can’t beat an average error of 0.0606, then we need to rethink our approach.


**TRAIN MODEL**

After all the work of data preparation, creating and training the model is pretty simple using Scikit-learn. We import the random forest regression model from skicit-learn, instantiate the model, and fit (scikit-learn’s name for training) the model on the training data. (Again setting the random state for reproducible results). This entire process is only 3 lines in scikit-learn!

In [ ]:
#TRAIN MODEL
#Don't really know what values like R^2 Training Score means
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels);
print('R^2 Training Score: {:.2f} \nR^2 Testing Score: {:.2f}'.format(rf.score(train_features, train_labels), 
                                                                                             rf.score(test_features, test_labels)))
#print('R^2 Training Score: {:.2f} \nOOB Score: {:.2f} \nR^2 Testing Score: {:.2f}'.format(rf.score(train_features, train_labels), 
                                                                                             #rf.oob_score_,
                                                                                             #rf.score(test_features, test_labels)))

R^2 Training Score: 0.25 
R^2 Testing Score: 0.22


**MAKE PREDICTIONS ON THE TEST SET**

Our model has now been trained to learn the relationships between the features and the targets. The next step is figuring out how good the model is! To do this we make predictions on the test features (the model is never allowed to see the test answers). We then compare the predictions to the known answers. When performing regression, we need to make sure to use the absolute error because we expect some of our answers to be low and some to be high. We are interested in how far away our average prediction is from the actual value so we take the absolute value (as we also did when establishing the baseline).

In [ ]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
#print(predictions)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
print(errors)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 4))

[1.45424684 1.67481478 1.55715537 ... 1.57269057 2.11628589 0.59706956]
Mean Absolute Error: 1.3403


In [ ]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / abs(test_labels))
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')


Accuracy: 81.19 %.


Our average estimate is off by 0.0003. compare to average improvement over the baseline that is 0.0606. Having a 6% difference

**Interpret Model and Report Results**

At this point, we know our model is good, but it’s pretty much a black box. We feed in some Numpy arrays for training, ask it to make a prediction, evaluate the predictions, and see that they are reasonable. The question is: how does this model arrive at the values? There are two approaches to get under the hood of the random forest: first, we can look at a single tree in the forest, and second, we can look at the feature importances of our explanatory variables.

**Visualizing a Single Decision Tree**

One of the coolest parts of the Random Forest implementation in Skicit-learn is we can actually examine any of the trees in the forest. We will select one tree, and save the whole tree as an image.
The following code takes one tree from the forest and saves it as an image.

In [ ]:
# Import tools needed for visualization
#from sklearn.tree import export_graphviz
#import pydot

# Pull out one tree from the forest
tree = rf.estimators_[5]
# Import tools needed for visualization
#from sklearn.tree import export_graphviz
#import pydot

# Pull out one tree from the forest
tree = rf.estimators_[5]

# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)

# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')

# Write graph to a png file
graph.write_png('tree.png')

# Display in python
#import matplotlib.pyplot as plt
plt.figure(figsize = (50, 50))
plt.imshow(plt.imread('tree.png'))
plt.axis('off');
plt.show();

# Download tree
from google.colab import files
plt.savefig('tree.png')
files.download('tree.png') 





In [ ]:
# Limit depth of tree to 3 levels
rf_small = RandomForestRegressor(n_estimators=10, max_depth = 3)
rf_small.fit(train_features, train_labels)

# Extract the small tree
tree_small = rf_small.estimators_[5]

# Save the tree as a png image
export_graphviz(tree_small, out_file = 'small_tree.dot', feature_names = feature_list, rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('small_tree.dot')
graph.write_png('small_tree.png');

# Display in python
#import matplotlib.pyplot as plt
plt.figure(figsize = (25, 25))
plt.imshow(plt.imread('small_tree.png'))
plt.axis('off');
plt.show();

# Download tree
from google.colab import files
plt.savefig('small_tree.png')
files.download('small_tree.png') 



**Variable Importances**

In order to quantify the usefulness of all the variables in the entire random forest, we can look at the relative importances of the variables. The importances returned in Skicit-learn represent how much including a particular variable improves the prediction. The actual calculation of the importance is beyond the scope of this post, but we can use the numbers to make relative comparisons between variables.
The code here takes advantage of a number of tricks in the Python language, namely list comprehensive, zip, sorting, and argument unpacking. It’s not that important to understand these at the moment, but if you want to become skilled at Python, these are tools you should have in your arsenal!

In [ ]:
# Get numerical feature importances
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 4)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

In [ ]:
# Didn't run this section of code cus all of our vairables have some kind of importance in predicting cost -Madu
# New random forest with only the two most important variables
rf_most_important = RandomForestRegressor(n_estimators = 1000 , n_jobs = -1, oob_score = True, bootstrap= True, random_state = 42)

# Extract the two most important features
important_indices = [feature_list.index('Avg_Mutation_Freq'), feature_list.index('pos')]
train_important = train_features[:, important_indices]
test_important = test_features[:, important_indices]

# Train the random forest
rf_most_important.fit(train_important, train_labels)
print('R^2 Training Score: {:.2f} \nOOB Score: {:.2f} \nR^2 Testing Score: {:.2f}'.format(rf_most_important.score(train_important, train_labels), 
                                                                                             rf_most_important.oob_score_,
                                                                                             rf_most_important.score(test_important, test_labels)))

# Make predictions and determine the error
predictions = rf_most_important.predict(test_important)
errors = abs(predictions - test_labels)

# Display the performance metrics
print('Mean Absolute Error:', round(np.mean(errors), 4))



**Visualizations**

The first chart I’ll make is a simple bar plot of the feature importances to illustrate the disparities in the relative significance of the variables. Plotting in Python is kind of non-intuitive, and I end up looking up almost everything on Stack Overflow when I make graphs. Don’t worry if the code here doesn’t quite make sense, sometimes fully understanding the code isn’t necessary to get the end result you want!

In [ ]:
# Import matplotlib for plotting and use magic command for Jupyter Notebooks
#import matplotlib.pyplot as plt
#%matplotlib inline
# Set the style
plt.style.use('fivethirtyeight')

# list of x locations for plotting
x_values = list(range(len(importances)))

# Make a bar chart
plt.bar(x_values, importances, orientation = 'vertical')

# Tick labels for x axis
plt.xticks(x_values, feature_list, rotation='vertical')

# Axis labels and title
plt.ylabel('Importance'); plt.xlabel('Features'); plt.title('Features Importances');